In [ ]:
using Pkg
Pkg.activate("../..")
Pkg.instantiate()

# Batagelj-Zaveršnik k-core decomposition algorithm

In [ ]:
using Graphs

function batagelj_zaversnik_k_core(g::SimpleGraph)
  n_v = nv(g)
  D = zeros(Int64, n_v)
  pos = zeros(Int64, n_v)
  vert = zeros(Int64, n_v)
  md = 0
  for v in 1:n_v
    D[v] = degree(g, v)
    md = max(md, D[v])
  end

  bin = zeros(Int64, md)

  for v in 1:n_v
    bin[D[v]] += 1
  end

  start = 1
  for d in 1:md
    num = bin[d]
    bin[d] = start
    start += num
  end

  for v in 1:n_v
    pos[v] = bin[D[v]]
    vert[pos[v]] = v
    bin[D[v]] += 1
  end

  for d in md:-1:2
    bin[d] = bin[d-1]
  end
  bin[1] = 1

  for i in 1:n_v
    v = vert[i]
    for u in neighbors(g, v)
      if D[u] > D[v]
        du = D[u]
        pu = pos[u]
        pw = bin[du]
        w = vert[pw]
        if u != w
          pos[u] = pw
          pos[w] = pu
          vert[pu] = w
          vert[pw] = u
        end
        bin[du] += 1
        D[u] -= 1
      end
    end
  end

  D
end

## Create network from Batagelj-Zaveršnik k-core array

In [ ]:
using Graphs

function batagelj_zaversnik_k_core_as_network(g::SimpleGraph, k::Int64)
  indices = batagelj_zaversnik_k_core(g)
  # create a vector of vertices that are in the k-core
  induction_vertices = [i for i in 1:length(indices) if indices[i] >= k]
  # create a subgraph of the original graph with only the vertices in the k-core
  g2, vmap = induced_subgraph(g, induction_vertices)
  g2
end

## Straight forward algoritam za dekompoziciju

Pravi novu mrežu koja sadrži samo čvorove u k-core-u

In [ ]:
using Graphs

function straight_forward_k_core(g::SimpleGraph, k::Int64)
  if nv(g) == 0
    return g
  end

  # copy the graph
  g2 = SimpleGraph()
  for i in 1:nv(g)
    add_vertex!(g2)
  end
  for e in edges(g)
    add_edge!(g2, e.src, e.dst)
  end

  # ε is the list of vertices in the k-core
  ε = vertices(g2)
  # remove vertices with degree greater than k
  achieved_k_core = true
  while true
    # vertices that will be removed
    δ = [i for i in 1:nv(g2) if degree(g2, i) < k]
    # if δ is empty, k-core network is found
    if length(δ) == 0
      break
    end
    # remove vertices from δ in g2
    rem_vertices!(g2, δ)
  end

  g2
end

In [ ]:
using Graphs

g = barabasi_albert_simple_graph(100, 0.1, 200, 5)

sf = straight_forward_k_core(g, 6)
bz = batagelj_zaversnik_k_core_as_network(g, 6)
@show sf
@show bz

In [ ]:
using Graphs

g = barabasi_albert_simple_graph(100, 0.1, 200, 5)

g2, vmap = batagelj_zaversnik_k_core_as_network(g, 5)

@info nv(g)
@info nv(g2)

# Simple test graphs (15-20 nodes)

Graphs are generated by a human. They are hardcoded here.

In [ ]:
using Graphs

small_graph_1 = SimpleGraph()

for i in 1:18
  add_vertex!(small_graph_1)
end

edge_pairs_1 = [
  (1, 2),
  (1, 3),
  (2, 3),
  (2, 8),
  (4, 7),
  (4, 8),
  (5, 6),
  (6, 7),
  (7, 8),
  (7, 10),
  (7, 11),
  (7, 12),
  (7, 13),
  (7, 15),
  (8, 9),
  (8, 10),
  (8, 12),
  (9, 16),
  (12, 14),
  (14, 18),
  (16, 17),
]

for (i, j) in edge_pairs_1
  add_edge!(small_graph_1, i, j)
end

k = maximum(core_number(small_graph_1))

@show k_core(small_graph_1, k)
@show vertices(batagelj_zaversnik_k_core_as_network(small_graph_1, k))
@show vertices(straight_forward_k_core(small_graph_1, k))

Broj čvorova koji pripadaju k-core-u na osnovu Graphs.jl implementacije je isti kao i u naše dve implementacije (Batagelj-Zaveršnik i straight forward implementacija).

In [ ]:
using Graphs

small_graph_2 = SimpleGraph()

for i in 1:17
  add_vertex!(small_graph_2)
end

edge_pairs_2 = [
  (1, 2),
  (1, 3),
  (2, 3),
  (3, 4),
  (4, 7),
  (4, 8),
  (4, 10),
  (5, 11),
  (6, 7),
  (7, 8),
  (7, 9),
  (7, 10),
  (7, 14),
  (7, 16),
  (8, 10),
  (8, 11),
  (9, 13),
  (10, 15),
  (10, 17),
  (11, 12),
]

for (i, j) in edge_pairs_2
  add_edge!(small_graph_2, i, j)
end

k = maximum(core_number(small_graph_2))

@show k_core(small_graph_2, k)
@show vertices(batagelj_zaversnik_k_core_as_network(small_graph_2, k))
@show vertices(straight_forward_k_core(small_graph_2, k))

In [ ]:
using Graphs

small_graph_3 = SimpleGraph()

for i in 1:23
  add_vertex!(small_graph_3)
end

egde_pairs_3 = [
  (1, 7),
  (2, 7),
  (3, 7),
  (3, 8),
  (3, 12),
  (4, 5),
  (5, 9),
  (6, 7),
  (6, 12),
  (7, 12),
  (8, 12),
  (9, 13),
  (10, 11),
  (11, 12),
  (11, 15),
  (11, 16),
  (12, 13),
  (12, 16),
  (12, 17),
  (12, 18),
  (13, 14),
  (13, 18),
  (15, 19),
  (15, 20),
  (16, 21),
  (17, 21),
  (18, 22),
  (18, 23),
]

for (i, j) in egde_pairs_3
  add_edge!(small_graph_3, i, j)
end

k = maximum(core_number(small_graph_3))

@show k_core(small_graph_3, k)
@show vertices(batagelj_zaversnik_k_core_as_network(small_graph_3, k))
@show vertices(straight_forward_k_core(small_graph_3, k))

In [ ]:
using Graphs

large_graph_1 = SimpleGraph(100, 1650)

println(k_core(large_graph_1))
println(batagelj_zaversnik_k_core(large_graph_1))

In [ ]:
using Graphs
large_graph_2 = SimpleGraph(1000, 166500)

println(k_core(large_graph_2))
println(batagelj_zaversnik_k_core(large_graph_2))

In [ ]:
rand(1:10, 5)

## Erdos-Renyi model

In [ ]:
using Random

function erdos_renyi_simple_graph(vs::Int64, es::Int64)
  S = Iterators.product(1:vs, 1:vs) |> collect |> filter(x -> x[1] != x[2])
  Random.shuffle!(S)

  G = SimpleGraph(vs)

  for (i, j) in S[1:es]
    add_edge!(G, i, j)
  end

  G
end

In [ ]:
using Graphs

erdos_renyi_graph_1 = erdos_renyi_simple_graph(10000, 165000)

k = maximum(core_number(erdos_renyi_graph_1))

@show k_core(erdos_renyi_graph_1, k)
@show vertices(batagelj_zaversnik_k_core_as_network(erdos_renyi_graph_1, k))
@show vertices(straight_forward_k_core(erdos_renyi_graph_1, k))

## Gilbertov model

In [ ]:
using Graphs

function gilbert_simple_graph(vs::Int64, p::Float64)
  G = SimpleGraph(vs)

  for i in 1:vs-1
    for j in i+1:vs
      if rand() < p
        add_edge!(G, i, j)
      end
    end
  end
  
  G
end

In [ ]:
using Graphs

gilbert_graph_1 = gilbert_simple_graph(10000, 0.0165)

k = maximum(core_number(gilbert_graph_1))

@show k_core(gilbert_graph_1, k)
@show vertices(batagelj_zaversnik_k_core_as_network(gilbert_graph_1, k))
@show vertices(straight_forward_k_core(gilbert_graph_1, k))

## Barabaši-Albert model

In [ ]:
using Graphs

function barabasi_albert_simple_graph(m0::Int64, p::Float64, N::Int64, m::Int64)
  G = gilbert_simple_graph(m0, p)

  Δ = []
  for i in 1:m0
    append!(Δ, repeat([i], degree(G, i)))
  end

  for i in m0+1:N
    add_vertex!(G)

    for j in 1:m
      v = rand(Δ)
      add_edge!(G, i, v)
      append!(Δ, [v])
    end

    append!(Δ, repeat([i], m))
  end

  G
end

In [ ]:
using Graphs

barabasi_albert_graph_1 = barabasi_albert_simple_graph(100, 0.1, 200, 5)

k = maximum(core_number(barabasi_albert_graph_1))

@show k_core(barabasi_albert_graph_1, k)
@show vertices(batagelj_zaversnik_k_core_as_network(barabasi_albert_graph_1, k))
@show vertices(straight_forward_k_core(barabasi_albert_graph_1, k))

In [ ]:
using Graphs
using Karnak
using NetworkLayout
using Colors

@drawsvg begin
  background("black")
  sethue("grey40")
  fontsize(8)
  drawgraph(barabasi_albert_graph_1,
    layout=stress,
    vertexlabels = vertices(barabasi_albert_graph_1),
    vertexfillcolors = 
      [RGB(rand(3)/2...)
        for i in vertices(barabasi_albert_graph_1)],
  )
end

In [ ]:
using Graphs

function stochastic_block_model_simple_graph(N::Int64, q::Int64, m::Vector{Float64}, B::Matrix{Float64})
  G = gilbert_simple_graph(N, 0.005)
  g = Vector{Int64}(undef, N)

  for i in 1:N
    r = rand()
    s = 0.0
    for k in 1:q
      s += m[k]
      if r <= s
        g[i] = k
        break
      end
    end
  end

  @show g

  for i in 1:N-1
    for j in i+1:N
      if rand() <= B[g[i], g[j]]
        add_edge!(G, i, j)
      end
    end
  end

  G
end

In [ ]:
using Graphs
using Karnak
using NetworkLayout
using Colors

stochastic_block_model_graph_1 = stochastic_block_model_simple_graph(100, 3, [0.33, 0.33, 0.34], [0.8 0.1 0.1; 0.1 0.8 0.1; 0.1 0.1 0.8])

@drawsvg begin
  background("black")
  sethue("grey40")
  fontsize(8)
  drawgraph(stochastic_block_model_graph_1,
    layout=stress,
    vertexlabels = vertices(stochastic_block_model_graph_1),
    vertexfillcolors = 
      [RGB(rand(3)/2...)
        for i in vertices(stochastic_block_model_graph_1)]
  )
end

# Facebook network

Nodes: 4039

Edges: 88234

Source: https://snap.stanford.edu/data/ego-Facebook.html

In [ ]:
using Graphs

all_files = readdir("facebook", join=true)

facebook_graph = SimpleGraph(4039)

for f in filter(f -> endswith(f, ".edges"), all_files)
  ego_v = parse(Int64, match(r"facebook/(\d+).edges", f).captures[1]) + 1
  for line in eachline(f)
    i, j = split(line, " ")
    i, j = parse(Int64, i) + 1, parse(Int64, j) + 1
    add_edge!(facebook_graph, i, j)
    add_edge!(facebook_graph, ego_v, i)
    add_edge!(facebook_graph, ego_v, j)
  end
end

for line in eachline("facebook/facebook_combined.txt")
  i, j = split(line, " ")
  add_edge!(facebook_graph, parse(Int64, i) + 1, parse(Int64, j) + 1)
end

In [ ]:
using Graphs

@show nv(facebook_graph)
@show ne(facebook_graph)

In [ ]:
using Graphs

core_num = core_number(facebook_graph)
deg = degree(facebook_graph)
betw = betweenness_centrality(facebook_graph)
clos = closeness_centrality(facebook_graph)

In [ ]:
using Statistics

# pairwise correlation using cartesian product
corelations = cor([core_num deg betw clos], dims=1)
cor()

In [ ]:
using Plots

# plot pairwise correlation as heatmap with correlation values
heatmap(["core number", "degree", "betweenness", "closeness"], ["core number", "degree", "betweenness", "closeness"], corelations, c=:blues)